# 📚 Notebook 01: Image Basics

Trong notebook này, bạn sẽ học:
1. Cách load và hiển thị ảnh
2. Image representation (pixels, channels)
3. Chuyển đổi giữa PIL, NumPy, và PyTorch tensors
4. Basic image manipulations

In [ ]:
# Import các thư viện cần thiết
import numpy as np
import torch
from PIL import Image
import matplotlib.pyplot as plt

# Kiểm tra phiên bản
print(f"PyTorch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 1. Tạo một ảnh đơn giản

In [ ]:
# Tạo ảnh RGB đơn giản
width, height = 256, 256

# Tạo gradient từ đỏ sang xanh
img_array = np.zeros((height, width, 3), dtype=np.uint8)

for y in range(height):
    for x in range(width):
        img_array[y, x, 0] = 255 - x  # Red channel
        img_array[y, x, 1] = x        # Green channel
        img_array[y, x, 2] = y        # Blue channel

# Hiển thị
plt.figure(figsize=(6, 6))
plt.imshow(img_array)
plt.title("Custom Gradient Image")
plt.axis('off')
plt.show()

print(f"Shape: {img_array.shape}")
print(f"Dtype: {img_array.dtype}")
print(f"Value range: [{img_array.min()}, {img_array.max()}]")

## 2. Chuyển đổi NumPy ↔ PyTorch

In [ ]:
# NumPy: (H, W, C) với giá trị 0-255
# PyTorch: (C, H, W) với giá trị 0-1

# NumPy -> PyTorch
tensor = torch.from_numpy(img_array).float() / 255.0  # Normalize to [0, 1]
tensor = tensor.permute(2, 0, 1)  # (H, W, C) -> (C, H, W)

print(f"NumPy shape: {img_array.shape}")
print(f"Tensor shape: {tensor.shape}")
print(f"Tensor range: [{tensor.min():.2f}, {tensor.max():.2f}]")

In [ ]:
# PyTorch -> NumPy (để hiển thị)
def tensor_to_display(t):
    """Convert PyTorch tensor to displayable numpy array."""
    arr = t.permute(1, 2, 0).numpy()  # (C, H, W) -> (H, W, C)
    arr = np.clip(arr, 0, 1)  # Clamp values
    return arr

plt.imshow(tensor_to_display(tensor))
plt.title("Image from Tensor")
plt.axis('off')
plt.show()

## 3. Thử thêm nhiễu (noise) vào ảnh

In [ ]:
# Thêm Gaussian noise
epsilon = 0.1  # Độ mạnh của nhiễu
noise = torch.randn_like(tensor) * epsilon

noisy_tensor = torch.clamp(tensor + noise, 0, 1)

# So sánh
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(tensor_to_display(tensor))
axes[0].set_title("Original")
axes[0].axis('off')

axes[1].imshow(tensor_to_display(noise + 0.5))  # Shift for visibility
axes[1].set_title(f"Noise (epsilon={epsilon})")
axes[1].axis('off')

axes[2].imshow(tensor_to_display(noisy_tensor))
axes[2].set_title("Noisy Image")
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 4. Thử với ảnh thực tế

In [ ]:
# Sử dụng module của chúng ta
import sys
sys.path.insert(0, '../src')

from ai_shield.core.image import load_image, tensor_to_image
from ai_shield.core.metrics import calculate_psnr, calculate_ssim

# Tạo ảnh test và lưu
test_img = Image.new('RGB', (256, 256), color='blue')
test_img.save('../examples/sample_images/test.png')

# Load it back
img_tensor = load_image('../examples/sample_images/test.png')
print(f"Loaded tensor shape: {img_tensor.shape}")

## 5. Đo chất lượng với PSNR và SSIM

In [ ]:
# Thử các mức epsilon khác nhau
epsilons = [0.01, 0.03, 0.05, 0.1]

fig, axes = plt.subplots(1, len(epsilons) + 1, figsize=(20, 4))

# Original
axes[0].imshow(tensor_to_display(tensor))
axes[0].set_title("Original")
axes[0].axis('off')

for i, eps in enumerate(epsilons):
    noise = torch.randn_like(tensor) * eps
    noisy = torch.clamp(tensor + noise, 0, 1)
    
    psnr = calculate_psnr(tensor, noisy)
    ssim = calculate_ssim(tensor, noisy)
    
    axes[i+1].imshow(tensor_to_display(noisy))
    axes[i+1].set_title(f"ε={eps}\nPSNR={psnr:.1f}dB\nSSIM={ssim:.3f}")
    axes[i+1].axis('off')

plt.tight_layout()
plt.show()

## 📝 Bài tập

1. Thử tạo ảnh với các pattern khác nhau (stripes, checkerboard)
2. Thử các loại noise khác: uniform noise, salt-and-pepper
3. Tìm epsilon lớn nhất mà SSIM vẫn > 0.95